In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as mp


In [1]:
#base bid should be equal to the equal to the median (=84) since this higher than the mode (=70) but lower than the mean (= 105). Using the mean won't be accurate because few outliers skew the mean to the right.
base_bid = 84


#lorenz linear bidding

base = 0
bids = 0
impressions = 0
clicks = 0
bidPrice = (rtb_train.loc[:,"bidprice"].mean()/1000)

for i in range(0,len(rtb_validation)):
    if budget > 0 and budget > (rtb_validation['payprice'].values[i]/1000):
        if bidPrice > (rtb_validation['bidprice'].values[i]/1000):
            impressions += 1
            bids += 1
            clicks += rtb_validation['click'].values[i]
            budget = budget - (rtb_validation['payprice'].values[i]/1000)
    else: 
        print("Budget is spent!")
        break
        
ctr = (clicks/impressions)*100
cpm = 6250/impressions
cpc = 6250/clicks
#Output for the alrogithm
print("Output")
print("Number of Bids:",bids)
print("Number of Impressions:",impressions)
print("Number of Clicks:", clicks)
print("CTR is:",ctr, "%")
print("CPM is:",cpm)
print("CPC is:",cpc)

NameError: name 'rtb_train' is not defined